## **TD 11 et 12 - Collecte, Traitement, et Analyse de données de réseaux sociaux**

##### *LEFEVRE Laura et LE CORRE Camille - LDD BI*


### I- Traitement des données

Dans cette première partie, nous récupérons les données, nous les nettoyons et les organisons, pour obtenir un DataFrame contenant les données qui nous intéressent pour la suite du projet.

In [10]:
# Importation des différents modules utilisés dans notre programme

import json
import re
import pandas as pd
import os
import random as rd
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

# Modules a installer
    #!pip install textblob
    #!pip install textblob-fr

Dans un premier temps, on définit une fonction qui va nous permettre de récupérer dans une variable, la liste des dictionnaires représentant les différents tweets contenu dans le fichier json.

In [11]:
def openFile(fileName):
    '''Fonction qui permet de lire et de recuperer un fichier json dans une variable data_dict'''
    
    with open(fileName, encoding='utf8') as json_data:
        data_dict = json.load(json_data)
        
        return data_dict

Par exemple, la commande suivante permet de récupérer dans la varible dic, la liste des tweets du fichier "versailles_tweets_100.json". On obtient une liste de dictionnaire et dic[0] nous permet de visualiser le premier tweet. On peut voir qu'il possède plusieurs éléments qui ne vont pas nous servir. Nous allons donc pouvoir nettoyer chacun des tweets, en gardant les informations essentiels (author_id, text, hastags...). De plus, on va pouvoir nettoyer le texte du tweet en supprimant tous les caractères spéciaux.

In [12]:
dic = openFile("versailles_tweets_100.json")
dic[0]

# changer nom

{'_id': '1421616335700824064',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 1,
  'quote_count': 0},
 'id': '1421616335700824064',
 'conversation_id': '1421616335700824064',
 'author_id': '1339914264522461187',
 'text': 'Goumin des éléphants joueurs la même fatigue même 😫 #twitter225',
 'geo': {'place_id': '00b8943291443c8c'},
 'lang': 'fr',
 'created_at': '2021-07-31T23:38:41.000Z',
 'entities': {'hashtags': [{'start': 52, 'end': 63, 'tag': 'twitter225'}]}}

On code les fonctions permettant le nettoyage :

1. Supprimer les emojis

In [13]:
def removeEmojis(data):
    '''Fonction qui permet de supprimer les emojis du text d'un tweet 
    (source : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)'''
    
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

2. Supprimer tous les caractères spéciaux

In [14]:
def cleanText(text):
    '''Fonction qui permet de nettoyer le texte d'un tweet a l'aide de regex'''
    
    text = text.lower()
    text = removeEmojis(text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', "", text)                            # suppression des URL
    text = re.sub(r'@([A-Za-z0-9_]+)', "", text)                                # suppresion des mentions
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"#([A-Za-z0-9_]+)", "", text)                                # suppresion des hashtags
    text = re.sub(r"[\,\!\?\%\(\)\/\"\&\+\#\$\£\%\:\.\@\-\n]", "", text)
    text = re.sub(r"[\é\è\ê]", "e", text)
    text = re.sub(r"[\ù]", "u", text)
    text = re.sub(r"[\à]", "a", text)
    text = re.sub(r"[\î\ï]", "i", text)
    text = re.sub(r"( )+", " ", text)
    text = re.sub(r"( )*$", "", text)
    text = re.sub(r"^( )", "", text)
    
    return text

Il faut maintenant créer la zone d'atterrissage des tweets. 
On créer donc une fonction qui va nous permettre d'ajouter les tweets dans un autre fichier (zone_atterrissage.json), semblable à l'initial, mais avec les textes des tweets nettoyés. 

In [16]:
def zoneAtterrissage(tweet):
    '''Fonction qui permet de creer la zone d'atterrisage des tweets. Elle ajoute dans un fichier zone d'atterrissage, un tweet en le nettoyant.'''
    
    # Nettoie le texte du tweet et remplace le texte initiale par le texte nettoye
    tweet_clean = cleanText(tweet['text'])
    tweet['text'] = tweet_clean
    
    # Si le fichier de la zone d'atterrissage n'existe pas, on le creer et on y ajoute le tweet
    if os.path.exists("zone_atterrissage.json") == False:
        with open("zone_atterrissage.json", "w") as fil:
            json.dump([tweet], fil)
    
    # Sinon, on recupere les tweets deja presents, et on y ajoute celui en cours de traitement
    else :
        li_tweet = openFile("zone_atterrissage.json")
        with open("zone_atterrissage.json", 'w') as filout :
            li_tweet.append(tweet)
            json.dump(li_tweet, filout)
            
    return
       

Il nous faut à présent une fonction qui va nous permettre de traiter les tweets que l'on a à disposition. On va donc envoyer chaque tweet dans la zone d'atterrissage, l'un après l'autre.

In [17]:
def traitement_nettoyage(liste_tweet:list):
    '''Cette fonction permet de traiter tous les tweets et de les envoyer dans la zone d'atterrissage'''

    for elt in liste_tweet:
        zoneAtterrissage(elt)
    
    return

On execute cette méthode en passant en paramètre le fichier json contenant les tweets. Après l'exécution on obtient un nouveau fichier json avec les tweets nettoyés.

In [19]:
traitement_nettoyage(dic)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

A présent, tous les tweets sont stockés dans un nouveau fichier (zone_atterrissage.json). Ils sont nettoyés et on va pouvoir créer une zone d'entrepot, sous forme de DataFrame. On va y stocker tous les tweets avec seulement les informations dont on a besoin.
On va donc créer un certain nombre de fonctions qui vont nous permettre de récupérer ces informations.

1. Une fonction nous permettant d'extraire le texte d'un tweet

In [20]:
def textExtract(tweet):
    '''Fonction qui retourne le text d'un tweet'''

    # On recupere le texte
    text = tweet.get("text")

    # S'il n'y as pas de text (exemple : c'etait une mention), on retourne None
    if text == '':
        return
    
    return text

2. Une fonction nous permettant d'extraire les hashtags d'un tweet

In [21]:
def listHashtags(tweet):
    ''' Fonction qui renvoie la liste de hashtags d'un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''

    list_hashtags = []
    if "entities" in tweet:
        if "hashtags" in tweet["entities"]:                # on vérifie qu'il y a au moins un hashtag
            for h in range(len(tweet["entities"]["hashtags"])):            # on parcourt la liste des hashtags
                list_hashtags.append(tweet["entities"]["hashtags"][h]["tag"])   
            
            return list_hashtags
    
    return

3. Une fonction nous permettant de récupérer les utilisateurs mentionnés

In [22]:
def listMentionedUsers(tweet):
    ''' Fonction qui renvoie la liste des utilisateurs mentionnés dans un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''

    list_mentioned_users = []
    
    if "entities" in tweet:
        if "mentions" in tweet["entities"]:                # on vérifie qu'il y a au moins un utilisateur mentionné
            for m in range(len(tweet["entities"]["mentions"])):            # on parcourt la liste des utilisateurs mentionnés
                list_mentioned_users.append(tweet["entities"]["mentions"][m]["username"])
            
            return list_mentioned_users
    return

4. Une fonction nous permettant d'attribuer un topic à un tweet

In [23]:
def topic(tweet):
    ''' Fonction qui attribue un topic à un tweet de façon aléatoire.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''
    
    topics = ['sport', 'politic', 'environment', 'TV', 'animal', 'nature', 'celebrity',
                                'travel', 'singing', 'love', 'fun', 'history', 'culture']
    
    if tweet.get('text') != '':             # si le tweet contient un texte
        return rd.choice(topics)            # on va lui attribuer un topic
    
    return                                  # sinon, aucun topic n'est attribué 

5. Une fonction nous permettant d'attribuer un sentiment au tweet (en fonction du texte)

In [24]:
# /!\ il faut trouver le paramètre pour l'adapter en français ou toutes les langues

def feeling(tweet):
    ''' Fonction qui détermine le sentiment d'un tweet (positif, négatif ou neutre).
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''
    
    feeling = TextBlob(tweet.get("text")).sentiment
    
    if feeling[0] < 0:
        return "negative"
    elif feeling[0] == 0:
        return "neutral"
    else:
        return "positive"

Après avoir déclaré toutes ces fonctions, on peut créer la fonction zoneEntrepot pour créer un DataFrame contenant les tweets avec les informations des fonctions précédentes. Chaque tweet va correspondre à une ligne.

In [25]:
def zoneEntrepot(list):
    '''Fonction qui prend en entrée la liste des dictionnaires correspondant
    aux tweets issus du fichier zone_atterrissage.json et qui créer un DataFrame
    contenant uniquement les informations qui nous intéressent'''

    # Création d'un DataFrame
    zone_entrepot = pd.DataFrame(columns = ['author_id', 'text', 'hashtags', 'mentioned_users', 'topics', 'feelings'])

    for k in range(len(list)):        # on travaille sur un tweet à la fois
        # Création d'une liste qui va contenir toutes les informations utiles de ce tweet
        tweet = []
        
        # Ajout à cette liste des informations récupérées de la zone d'atterissage
        tweet.extend([list[k].get("author_id"), textExtract(list[k]), listHashtags(list[k]), listMentionedUsers(list[k]),topic(list[k]), feeling(list[k])])  
        
        # Ajout de la ligne correspondant au tweet à notre DataFrame
        zone_entrepot.loc[k+1] = tweet

    return zone_entrepot    

In [ ]:
def traitement_entrepot(file:list):
    '''Fonction qui effectue le traitement d'une liste de tweets provenant de la zone d'atterrissage. Le traitement consiste à envoyer 
    les tweets dans une zone d'entrepot avec les informations essentielles'''


In [26]:
liste_tweet = openFile("zone_atterrissage.json")
zone_entreport_tweet = zoneEntrepot(liste_tweet)
zone_entreport_tweet

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
# zoneEntrepot version Cam

def ZoneEntrepot(list):
    '''Fonction qui prend en entrée la liste des dictionnaires correspondant
    aux tweets issus du fichier zone_atterrissage et qui créer un DataFrame
    contenant uniquement les informations qui nous intéressent'''

    # Création d'un DataFrame (chaque ligne va correspondre à un tweet)
    zone_entrepot = pd.DataFrame(columns = ['author_id', 'text', 'hashtags', 'mentioned_users', 'topics', 'feelings'])

    for k in range(len(list)-1):        # on travaille sur un tweet à la fois
        # Création d'une liste qui va contenir toutes les informations utiles de ce tweet
        tweet = []
        # Ajout à cette liste des informations récupérées de la zone d'atterissage
        tweet.append(list[k].get("author_id"))              # auteur du tweet    
        tweet.append(list[k].get("text"))                   # texte du tweet
        tweet.append(listHashtags(list[k]))                 # liste des hashtags
        tweet.append(listMentionedUsers(list[k]))           # liste des utilisateurs mentionnés
        tweet.append(topic(list[k]))                        # topic
        tweet.append(feeling(list[k]))                      # sentiment
        
        # Ajout de la ligne correspondant à ce tweet à notre DataFrame
        zone_entrepot.loc[k+1] = tweet

    return zone_entrepot    

### II - Analyse des données

Maintenant que nos tweets ont été nettoyés et que nous avons un DataFrame contenant uniquement les informations qui nous intéressent, nous allons pouvoir analyser et effectuer des statistiques sur nos tweets.

Tout d'abord, nous cherchons à obtenir les utilisateurs publiant le plus de tweets, les hashtags ou les mentions les plus utilisés, ou bien les topics les plus répandus. Les fonctions suivantes vont permettre à l'utilisateur de connaître le "Top k" parmis ces informations, où k est un nombre choisi par ce dernier.

Les deux fonctions suivantes seront utilisées pour obtenir les tops k :

In [7]:
def listColumn(df, column:str):
    ''' Fonction qui renvoie les éléments d'une colonne d'un DataFrame sous forme d'une liste'''

    type_elem = type(df[column][1])                     # on stocke le type des éléments de la colonne dans une variable

    if type_elem == int or type_elem == str:            # si la colonne contient des entiers ou une chaîne de caractères
        return list(df[column])
    elif type_elem == list:                             # si la colonne contient des listes (une liste de hashatgs
                                                        # ou d'utilisateurs mentionnés par exemple)
        l_lists = list(df[column])
        l_elem = []
        for e in l_lists:
            for ee in e:
                l_elem.append(ee)
        return l_elem

In [8]:
def nbOccurencesIntoDict(list_column):
    ''' Fonction qui crée un dictionnaire dans lequel chaque clé correspond à un utilisateur
    ou un hashtag etc, et chaque valeur correspond au nombre d'occurrences de cet élément dans la liste'''

    dic = {}

    for elem in list_column:                # on parcourt la liste de tous les éléments d'une colonne
        # Si on a déjà rencontré cet élément, on incrémente son nombre d'occurences de 1
        if elem in dic:                     
            dic[elem] += 1
        # Si on ne l'a jamais rencontré encore, on ajoute une clé               
        else:
            dic[elem] = 1

    return dic

Les quatres fonctions suivantes permettent de trier les hashtags, les utilisateurs, les utilisateurs mentionnés ou bien les topics, que l'on retrouve le plus de fois dans la colonne du DataFrame correspondante, à celui qu'on retrouve le moins de fois.

1. Pour les hashtags

In [27]:
def topKHashtags(df, k):
    ''' Fonction renvoyant les k hashtags les plus utilisés'''

    # On créer un dictionnaire contenant comme clés les différents hashtags
    # et comme valeur le nombre de fois que chaque hashtag a été utilisé
    dic_occurences = nbOccurencesIntoDict(listColumn(df, 'hashtags'))

    # On trie le dictionnaire par ordre croissant des valeurs et on le transforme
    # en liste pour renvoyer les k premiers hashtags les plus utilisés
    return list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))[:k]

In [ ]:
def sortHashtags(df):
    ''' Fonction renvoyant une liste des hashtags triés en fonction
    du nombre de fois qu'ils ont été utilisés'''

    # On créer un dictionnaire contenant comme clés les différents hashtags
    # et comme valeur le nombre de fois que chaque hashtag a été utilisé
    dic_occurences = nbOccurencesIntoDict(listColumn(df, 'hashtags'))

    # On trie le dictionnaire par ordre croissant des valeurs et on le transforme en liste
    l = list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))

    return l

2. Pour les utilisateurs

In [28]:
def topKUsers(df, k):
    ''' Fonction renvoyant les k utilisateurs ayant publié le plus de tweets'''

    # On créer un dictionnaire contenant comme clés les différents utilisateurs et
    # comme valeur le nombre de tweets qu'ils ont publié
    dic_occurences = nbOccurencesIntoDict(listColumn(df, "author_id"))

    # On renvoie une liste des k utilisateurs ayant publié le plus de tweets
    return list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))[:k]

In [ ]:
def sortUsers(df):
    ''' Fonction renvoyant une liste des utilisateurs triés en fonction
    du nombre de fois qu'ils ont publié un tweet'''

    # On créer un dictionnaire contenant comme clés les différents utilisateurs et
    # comme valeur le nombre de tweets qu'ils ont publié
    dic_occurences = nbOccurencesIntoDict(listColumn(df, 'author_id'))

    # On trie le dictionnaire par ordre croissant des valeurs et on le transforme en liste
    l = list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))

    return l

3. Pour les utilisateurs mentionés

In [30]:
def topKMentionedUsers(df, k):
    ''' Fonction renvoyant les k utilisateurs les plus mentionnés'''

    # On créer un dictionnaire contenant comme clés les différents utilisateurs
    # mentionnés et comme valeur le nombre de fois qu'ils ont été mentionés
    dic_occurences = nbOccurencesIntoDict(listColumn(df, "mentioned_users"))

    # On renvoie une liste des k utilisateurs mentionés le plus de fois
    return list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))[:k]    

In [ ]:
def sortMentionedUsers(df):
    ''' Fonction renvoyant une liste des utilisateurs mentionnés
    triés en fonction du nombre de mentions'''

    # On créer un dictionnaire contenant comme clés les différents utilisateurs
    # mentionnés et comme valeur le nombre de fois qu'ils ont été mentionés
    dic_occurences = nbOccurencesIntoDict(listColumn(df, 'mentioned_users'))

    # On trie le dictionnaire par ordre croissant des valeurs et on le transforme en liste
    l = list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))

    return l

4. Pour les topics

In [29]:
# il faut gérer le cas quand les tweets n'ont pas de topic (None) -> dans liste colonne ? à discuter





def topKTopics(df, k):
    ''' Fonction renvoyant les k topics les plus répandus'''

    # On créer un dictionnaire contenant comme clés les différents
    # topics et comme valeur le nombre de tweets sur ce topic
    dic_occurences = nbOccurencesIntoDict(listColumn(df, "topics"))

    # On renvoie une liste des k topics les plus répandus
    l_sorted = list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))[:k]
    return 

In [ ]:
def sortTopics(df):
    ''' Fonction renvoyant une liste des topics triés en fonction
    du nombre de fois qu'ils ont été utilisés'''

    # On créer un dictionnaire contenant comme clés les différents
    # topics et comme valeur le nombre de tweets sur ce topic
    dic_occurences = nbOccurencesIntoDict(listColumn(df, 'topics'))

    # On trie le dictionnaire par ordre croissant des valeurs et on le transforme en liste
    l = list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))

    return l

Ces quatre fonctions sont utilisées dans la fonction qui suit, permettant à l'utilisateur de connaitre le Top k qu'il souhaite. 

In [31]:
def topK(df):
    ''' Fonction qui demande à l'utilisateur le Top k qu'il souhaite connaitre,
    ainsi que ce nombre k, puis qui renvoie ce classement demandé'''

    possible_top = ('H', 'U', 'UM', 'T')
    top = 0
    k = -1

    # On demande à l'utilisateur le Top qu'il souhaite connaitre
    while top not in possible_top:
        top = str(input("Quel Top voulez-vous connaitre ? hashtags (tapez H), utilisateurs (tapez U), utilisateurs mentionnés (tapez UM) ou bien topics (tapez T) ?"))
    
    # Utilisation des fonctions de tri par ordre croissant des colonnes
    # du DataFrame en fonction du choix de l'utilisateur
    if top == 'H':
        l = sortHashtags(df)
    elif top == 'U':
        l = sortUsers(df)
    elif top == 'UM':
        l = sortMentionedUsers(df)
    else:
        l = sortTopics(df)

    # Nombre maximum que l'utilisateur pourra entrer pour la valeur de k
    k_max = len(l)
    
    # Choix de la valeur k par l'utilisateur
    while (k < 0) and (k > k_max):
        k = int(input("Veuillez saisir k : "))
    
    return l[:k]

Pour tester cette fonction et connaître le top k de votre choix, vous pouvez executer la cellule suivante et vous laisser guider par les instructions :

In [ ]:
print(topK(dic))